In [1]:
# importing the modules that will be needed
from collections import defaultdict
import csv 
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import pprint
import re
import os
import codecs
import schema

OSM_FILE = "map_PHX_Metro.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

In [2]:
# Create Sample File

#!/usr/bin/env python
# -*- coding: utf-8 -*-

k = 140 # Parameter: take every k-th top level element 140 for 10k file

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')
    


In [3]:
#OSM_FILE = SAMPLE_FILE

In [4]:
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict


# File
OSM_FILE = SAMPLE_FILE 


# Iterative Parsing
# Count Tags
def count_tags(filename):
    tags = {}
    for _, elem in ET.iterparse(filename):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] += 1
    return tags


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(element.attrib['k']):
                keys['lower'] = keys['lower'] + 1
            elif lower_colon.search(element.attrib['k']):
                keys['lower_colon'] = keys['lower_colon'] + 1
            elif problemchars.search(element.attrib['k']):
                keys['problemchars'] = keys['problemchars'] + 1
            else:
                keys['other'] = keys['other'] + 1
    
    return keys

def key_count(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


# Test count tags function
def test():
    tags = count_tags(OSM_FILE)
    keys = key_count(OSM_FILE)
    
    pprint.pprint(tags)
    pprint.pprint(keys)	

    
test()

{'member': 570,
 'nd': 52121,
 'node': 43512,
 'osm': 1,
 'relation': 53,
 'tag': 31296,
 'way': 5974}
{'lower': 19122, 'lower_colon': 11005, 'other': 1169, 'problemchars': 0}


In [5]:
# Audit file

# Audit street names------------------------------------------------------------------------------
# Regular expression to check for characters at end of string, including optional period.
# Eg "Street" or "St."

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)

# Common street names
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Road", "Parkway", "Freeway", "Close", "Highway", "Circle", "Trail", "US", "60"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)



def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

# Iterate over the osmfile and create a dictionary mapping from expected street names
# to collected streets.
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    osm_file.close()
    return street_types   

            

In [6]:
# Test audit function
audit(OSM_FILE)

defaultdict(set,
            {'Date': {'North Date'},
             'Ki': {'West Gu u Ki'},
             'Ranchos': {'East Camino de Los Ranchos'},
             'Sol': {'West Camino del Sol'},
             'South': {'East Paradise Village Parkway South'},
             'Stewart': {'North Stewart'},
             'Sunnyvale': {'South Sunnyvale'},
             'Terrace': {'East Sandra Terrace'},
             'Verde': {'South Verde'},
             'Way': {'East Janice Way',
              'East Joseph Way',
              'North 102nd Way',
              'North 14th Way',
              'North 17th Way',
              'North 19th Way',
              'North 21st Way',
              'North 34th Way',
              'North 36th Way',
              'North 37th Way',
              'North 39th Way',
              'North 43rd Way',
              'North 55th Way',
              'North 60th Way',
              'North 7th Way',
              'North Escobar Way',
              'South Larkspur Way',
       

In [7]:
# Mapping for names to be updated
mapping = { " St ": "Street",
           "St": "Street",
            "St. ": "Street",
            "Blvd": "Boulevard",
            "Ct": "Court",
            "Dr, ": "Drive ",
            " Dr ": "Drive ",
            "Dr. ": "Drive ",
            "Rd": "Road",
            "Rd ": "Road",
            "Rd.": "Road",
            "Pl": "Place",
            "Ave": "Avenue ",
            "Ave.": "Avenue",
            "ln " : "Lane",            
            "S ": "South ",
            "S. ": " South ",
            "N ": " North ",
            "N. ": " North ",
            "W " : " West ",
            "W. ": " West ",
            "E ": "East ",
            "E. ": "East ",
            "Hwy ":"Highway ", 
            "S L": "South L",
            "n Bon": "North Bon",           
            ", Superior, AZ 85173": "West US-60",
            "5810 A":"A",
            "0 w D":"0 West D"            
            }

# Improving Street names
def update_name(name, mapping):
    for key in mapping.iterkeys():
        if re.search(key, name):
            name = re.sub(key, mapping[key], name)

    return name

def improve_street_name():
    st_types = audit(OSM_FILE)   

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)            
            print name, "=>", better_name         
       
            #Second Check replace bad street names with corrected ones       
            if True: 
                
                better_name = better_name.replace("Road.", "Road")
                better_name = better_name.replace("393 ", "")
                better_name = better_name.replace(", Chandler, AZ 85225", "")
                better_name = better_name.replace(" nueida", "")
                better_name = better_name.replace(" nue", "")
                
                better_name = better_name.replace(" nue#158.", "")
                better_name = better_name.replace(" ue`", "")
                better_name = better_name.replace("  nueue", "")
                better_name = better_name.replace("  nueue'", "")
                better_name = better_name.replace("Streetart", "Street")
                
                better_name = better_name.replace("Streete 4", "")
                better_name = better_name.replace("#158", "")                
                better_name = better_name.replace(" #100", "")
                better_name = better_name.replace(", Suite A", "")
                better_name = better_name.replace(", Building B", "")
                better_name = better_name.replace(" #B", "")
                better_name = better_name.replace(" C1", "")
                better_name = better_name.replace(", Suite 34", "")
                better_name = better_name.replace(" #900", "")
                better_name = better_name.replace(". #1205", "")
                better_name = better_name.replace(" Suite D", "")
               
                better_name = better_name.replace("West US-60West US-60", "West US-60")
                better_name = better_name.replace("a Dr", "a Drive") 
                better_name = better_name.replace("e Dr", "e Drive")
                better_name = better_name.replace("t Dr", "t Drive")
                better_name = better_name.replace("y Dr", "y Drive")
                better_name = better_name.replace("x Dr", "x Drive")
                better_name = better_name.replace(" Dr.", " Drive")
                better_name = better_name.replace("Circle Driveive", "Circle Drive")
                better_name = better_name.replace("Vista Driveive", "Vista Drive")
                better_name = better_name.replace("Village Driveive", "Village Drive")
                better_name = better_name.replace("Norte Dr.", "Norte Drive")
                better_name = better_name.replace(" Streetick", " Stick")
                better_name = better_name.replace(" Pkwy", " Parkway")               
                better_name = better_name.replace(" Stadium", " Stadium")
                better_name = better_name.replace(" lane", " Lane")
                print name, "=>", better_name 
                
            #Third Check replace bad street names with corrected ones       
            if True:
                better_name = better_name.replace(" Avenueue`", " Avenue")
                better_name = better_name.replace(" Avenueue", " Avenue")
                better_name = better_name.replace(" Streetewart", " Stewart")                
                print name, "=>", better_name 

  

  

In [8]:
# Clean streets 
improve_street_name()

North 55th Way => North 55th Way
North 55th Way => North 55th Way
North 55th Way => North 55th Way
North 14th Way => North 14th Way
North 14th Way => North 14th Way
North 14th Way => North 14th Way
East Joseph Way => East Joseph Way
East Joseph Way => East Joseph Way
East Joseph Way => East Joseph Way
South Larkspur Way => South Larkspur Way
South Larkspur Way => South Larkspur Way
South Larkspur Way => South Larkspur Way
North 17th Way => North 17th Way
North 17th Way => North 17th Way
North 17th Way => North 17th Way
North 43rd Way => North 43rd Way
North 43rd Way => North 43rd Way
North 43rd Way => North 43rd Way
North 37th Way => North 37th Way
North 37th Way => North 37th Way
North 37th Way => North 37th Way
North 21st Way => North 21st Way
North 21st Way => North 21st Way
North 21st Way => North 21st Way
East Janice Way => East Janice Way
East Janice Way => East Janice Way
East Janice Way => East Janice Way
North 39th Way => North 39th Way
North 39th Way => North 39th Way
North 3

In [9]:
# Check Postalcodes for addresses 

# Regular expression to check whether postalcode is in appropriate format
postcode_re = re.compile('^[A-Z]{1,2}[0-9]{1,2}[A-Z]? [0-9][A-Z]{2}$') 

def is_postcode(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode")


# Search for postcodes within "way" and "node"
def find_postcode():
    osm_file = open(OSM_FILE, "r")
    postcode_types = set()
    odd_postcode = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postcode(tag):
                    m = postcode_re.search(tag.attrib['v'])
                    if m:
                        postcode_types.add(tag.attrib['v'])  
                    else:
                        odd_postcode.add(tag.attrib['v'])
                        

    osm_file.close()


    return (postcode_types, odd_postcode)



In [10]:
# Audit postal codes
find_postcode()

(set(),
 {'85003',
  '85004',
  '85006',
  '85007',
  '85007-4145',
  '85012',
  '85013',
  '85013-4408',
  '85015',
  '85015-3809',
  '85017',
  '85018',
  '85019',
  '85020',
  '85021',
  '85022',
  '85023',
  '85023-1508',
  '85023-2301',
  '85023-2510',
  '85023-8204',
  '85024',
  '85027',
  '85028',
  '85029',
  '85031',
  '85032',
  '85034',
  '85043',
  '85044',
  '85045',
  '85048',
  '85050',
  '85051',
  '85053',
  '85054',
  '850822',
  '85119',
  '85120',
  '85142',
  '85147',
  '85201',
  '85203',
  '85205',
  '85206',
  '85207',
  '85209',
  '85210',
  '85212',
  '85213',
  '85224',
  '85248',
  '85251',
  '85253',
  '85254',
  '85255',
  '85256',
  '85260',
  '85263',
  '85268',
  '85281',
  '85282',
  '85283',
  '85286',
  '85295',
  '85296',
  '85297',
  '85298',
  '85301',
  '85302',
  '85303',
  '85304',
  '85305',
  '85308',
  '85322',
  '85323',
  '85339',
  '85345',
  '85351',
  '85354',
  '85361',
  '85374',
  '85375',
  '85378',
  '85381',
  '85382',
  '85392',

In [11]:
# Noted a postalcode with a +4 number, will drop this and maintain only 9-digit postal code

area_postcode_re = re.compile('^[A-Z]{1,2}[0-9]{1,2}[A-Z]? ?[0-9]?$')

def update_postcode(odd_postcode):
    if area_postcode_re.search(odd_postcode):
        postcode = " "
    else:
        postcode = odd_postcode.split("-")[0]        
    return postcode
    


def improve_postcode():
    postcode_all = find_postcode()

    for postcode in postcode_all[1]:
        better_postcode = update_postcode(postcode)
        #print "1: ", postcode, "=>", better_postcode
        
        
        #Second Check replace bad zip codes with correct ones       
        if True:           
            better_postcode = better_postcode.replace("085028", "85028")
            better_postcode = better_postcode.replace("25248", "85248")
            better_postcode = better_postcode.replace("2804", "85204")
            better_postcode = better_postcode.replace("5015", "85015")
            better_postcode = better_postcode.replace("8502", "85020")
            better_postcode = better_postcode.replace("5015", "85015")     
            better_postcode = better_postcode.replace("82381", "85381")
            better_postcode = better_postcode.replace("82158", "85015")
            better_postcode = better_postcode.replace("84009", "85009")
            better_postcode = better_postcode.replace("8551", "85051")
            better_postcode = better_postcode.replace("85331;85377", "85331")
            better_postcode = better_postcode.replace("84017", "85017") 
            better_postcode = better_postcode.replace("850822", "85082") 
            #print "2: ", postcode, "=>", better_postcode
        
        #Third Check replace bad zip codes with correct ones       
        if True:
            better_postcode = better_postcode.replace("850203", "85023")
            better_postcode = better_postcode.replace("850204", "85024")
            better_postcode = better_postcode.replace("850201", "85021")
            better_postcode = better_postcode.replace("850200", "85020")
            better_postcode = better_postcode.replace("850202", "85022")
            better_postcode = better_postcode.replace("8885015", "85015")
            better_postcode = better_postcode.replace("885015", "85015")
            better_postcode = better_postcode.replace("850822", "85082")
            better_postcode = better_postcode.replace("850207", "85027")
            better_postcode = better_postcode.replace("850208", "85028")
            better_postcode = better_postcode.replace("850209", "85029")
            print "3: ", postcode, "=>", better_postcode
            

In [12]:
# Fix postal codes
improve_postcode()

3:  85392 => 85392
3:  85248 => 85248
3:  85023-8204 => 85023
3:  85396 => 85396
3:  85301 => 85301
3:  85303 => 85303
3:  85302 => 85302
3:  85305 => 85305
3:  85304 => 85304
3:  85308 => 85308
3:  85019 => 85019
3:  85018 => 85018
3:  85268 => 85268
3:  85013 => 85013
3:  85012 => 85012
3:  85015 => 85015
3:  85263 => 85263
3:  85017 => 85017
3:  850822 => 85082
3:  85023-2510 => 85023
3:  85120 => 85120
3:  85253 => 85253
3:  85044 => 85044
3:  85251 => 85251
3:  85006 => 85006
3:  85256 => 85256
3:  85004 => 85004
3:  85254 => 85254
3:  85034 => 85034
3:  85032 => 85032
3:  85031 => 85031
3:  85323 => 85323
3:  85322 => 85322
3:  85043 => 85043
3:  85003 => 85003
3:  85023-1508 => 85023
3:  85020 => 85020
3:  85021 => 85021
3:  85022 => 85022
3:  85023 => 85023
3:  85024 => 85024
3:  85023-2301 => 85023
3:  85027 => 85027
3:  85028 => 85028
3:  85029 => 85029
3:  85007 => 85007
3:  85255 => 85255
3:  85351 => 85351
3:  85224 => 85224
3:  85054 => 85054
3:  85051 => 85051
3:  85050 

In [13]:
import xml.etree.cElementTree as ET
import pprint
import re

def get_user(element):
    return

# Generates list of users
def process_users(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        name = element.attrib.get('user')
        if name != None:
            if name not in users:
                users.add(name)
                
    pass
    return users

In [14]:
# Check Users
process_users(OSM_FILE)

{'pluton_od',
 'scharith',
 'AndyAyre',
 'Nick Coury',
 'nairnidh',
 'CMOC',
 'nedluxaca',
 'ygudeti',
 'Sundance',
 'Brian@Brea',
 'ashleyannmathew',
 'West-MEC',
 'hereisam',
 'srsjojo',
 'chagasru',
 'ashradha',
 'Pixelcrunch',
 'frizatch',
 'fun60CxMR',
 'vasamd',
 'RichRico',
 'deejayh',
 'wallywall',
 'antimeridian',
 'soundred',
 'ammhita',
 'hemasvn',
 'ramyaragupathy',
 'syrah1',
 'LilyVilla',
 'Johnny Five',
 'Joe2oh',
 'bhavant',
 'sskalyan',
 'clay_c',
 'GeneBase',
 'LML013',
 'Mar Mar',
 'aurel_joys',
 'bradrh',
 'boiker',
 'bswitzman',
 'Kent Shaffer',
 'yurasi',
 'NicWow',
 'Pnrrth',
 'TomHoman',
 'tusnayak',
 'maxerickson',
 'tejajt',
 'tiwdeves',
 'jaywhyenecks',
 'RichAZICN',
 'Korgi1',
 'thetornado76',
 'sun0123',
 'harinikh',
 'Cuken',
 'MapperMudkip',
 'tomthepom',
 'racranji',
 'sawhs',
 'Adam Schneider',
 'pkkv',
 'harshjs',
 'MeltMyMind09',
 'IAmScottCarlson',
 'istvanv_telenav',
 'sunilvs',
 'Geo_Lugo',
 'pezizomycotina',
 'knalette',
 'nfgusedautoparts',
 'Eas

In [15]:
# Data to CSV
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from unittest import TestCase
import cerberus
import schema



# Make sure the fields order in the csvs matches the column order in the
# sql table schema
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# Shape each element into several data structures
# Clean and shape node or way XML element to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    node_attribs = {} 
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):  
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                node_tag = {} 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = tag.attrib['v']  

                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    node_tag['type'] = 'regular'
                    node_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+):', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])
                    node_tag['type'] = bef_colon[0]
                    node_tag['key'] = aft_colon[0]
                    if node_tag['type'] == "addr" and node_tag['key'] == "street":
                        # update street name
                        node_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif node_tag['type'] == "addr" and node_tag['key'] == "postcode":
                        # update post code
                        node_tag['value'] = update_postcode(tag.attrib['v']) 
            tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                way_tag = {}
                way_tag['id'] = element.attrib['id'] 
                way_tag['value'] = tag.attrib['v']
                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+?):+[a-z]', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])

                    way_tag['type'] = bef_colon[0]
                    way_tag['key'] = aft_colon[0]
                    if way_tag['type'] == "addr" and way_tag['key'] == "street":
                        way_tag['value'] = update_name(tag.attrib['v'], mapping) 
                    elif way_tag['type'] == "addr" and way_tag['key'] == "postcode":
                        way_tag['value'] = update_postcode(tag.attrib['v']) 
            tags.append(way_tag)
        position = 0
        for tag in element.iter("nd"):  
            nd = {}
            nd['id'] = element.attrib['id'] 
            nd['node_id'] = tag.attrib['ref'] 
            nd['position'] = position  
            position += 1
            
            way_nodes.append(nd)
    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}




# Helper Functions            

# Yield element if it is the right type of tag
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# Raise ValidationError if element does not match schema
def validate_element(element, validator, schema=SCHEMA):
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

# Extend csv.DictWriter to handle Unicode input
class UnicodeDictWriter(csv.DictWriter, object):
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# Main Function                    

# Iteratively process each XML element and write to csv(s)
def process_map(file_in, validate):

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()



        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_FILE, validate=False)

In [16]:
process_map(OSM_FILE, validate=False)

In [19]:
# Create DB
import csv, sqlite3

con = sqlite3.connect("PHX_AZ1.db")
con.text_factory = str
cur = con.cursor()

# create nodes table
cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#create nodes_tags table
cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

#Create ways table
cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#Create ways_nodes table
cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
con.commit()

#Create ways_tags table
cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

In [20]:
# Query DB
import csv, sqlite3

def number_of_nodes():
    result = cur.execute('SELECT COUNT(*) FROM nodes')
    return result.fetchone()[0]

def number_of_ways():
    result = cur.execute('SELECT COUNT(*) FROM ways')
    return result.fetchone()[0]

def number_of_Unique_users():
    result = cur.execute('SELECT COUNT(distinct(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)')
    return result.fetchone()[0]

def Top_Contributing_user():
    for row in cur.execute('SELECT e.user, COUNT(*) as num \
                            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e \
                            GROUP BY e.user \
                            ORDER BY num DESC \
                            LIMIT 1'):
        return row

def Biggest_religion():
    for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags \
                            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") i \
                            ON nodes_tags.id=i.id \
                            WHERE nodes_tags.key="religion" \
                            GROUP BY nodes_tags.value \
                            ORDER BY num DESC\
                            LIMIT 1'):
         return row

def popular_amenity():
    for row in cur.execute('SELECT value, COUNT(*) as num \
                            FROM nodes_tags \
                            WHERE key="amenity" \
                            GROUP BY value \
                            ORDER BY num DESC \
                            LIMIT 1'):
        return row

                
if __name__ == '__main__':

    con = sqlite3.connect("PHX_AZ1.db") 
    cur = con.cursor()
    print "Number of nodes: " , number_of_nodes()
    print "Number of ways: " , number_of_ways()
    print "Number of unique users: " , number_of_Unique_users()
    print "Top Contributing user: " , Top_Contributing_user()
    print "Biggest religion: " , Biggest_religion()
    print "popular amenity: " , popular_amenity()

Number of nodes:  43512
Number of ways:  5974
Number of unique users:  1231
Top Contributing user:  (u'Dr Kludge', 10705)
Biggest religion:  (u'christian', 3)
popular amenity:  (u'waste_disposal', 27)
